### <font color="blue"> Demo, direct interface </blue>

In [23]:
import ipyparallel

In [24]:
rc = ipyparallel.Client()

The `ids` attribute of Client instance shows identifiers of engines that IPython detected

In [25]:
rc.ids

[0, 1]

In [36]:
dview = rc[:]
dview

<DirectView [0, 1]>

In [30]:
dview2 = rc.direct_view()
dview2

<DirectView all>

In [32]:
rc.direct_view?

### <font color="blue"> Demo, %px magic </blue>

In [14]:
%px?

In [7]:
%px import os, time

In [8]:
%px print(time.time())

[stdout:0] 1481143070.13
[stdout:1] 1481143070.13


In [18]:
%px a = os.getpid()

In [19]:
print(a)

NameError: name 'a' is not defined

In [20]:
%px print(a)

[stdout:0] 62450
[stdout:1] 62453


In [15]:
%pxresult

[stdout:0] 62450
[stdout:1] 62453


In [21]:
%%px --targets :-1
    print(os.getpid())

[stdout:0] 62450


### <font color="red"> *Exercise: Estimating $\pi$ with parallel Monte Carlo* </font>

In [37]:
import ipyparallel

In [38]:
clients = ipyparallel.Client()
clients.ids

[0, 1]

In [39]:
dview = clients[:]
dview

<DirectView [0, 1]>

In [40]:
import numpy as np
def mc_pi(n):
#    import numpy as np
    count = 0
    p = np.random.rand(n,2)
    for i in range(n):
        x = p[i,0]
        y = p[i,1]
        if (x**2 + y**2) <= 1.0:
            count += 1
    return float(count)/float(n)

In [41]:
%%timeit
est_pi = mc_pi(10**6)
print(est_pi*4) 

3.14036
3.142328
3.13932
3.141224
1 loop, best of 3: 1.29 s per loop


May need to import modules on the engines

In [42]:
#dview.execute("import numpy as np")

In [43]:
%%timeit
results = dview.map_sync(mc_pi, [500000]*2)
print(np.sum(results))

1.57057
1.570804
1.569566
1.569978
1 loop, best of 3: 657 ms per loop


Speed up of factor 2 on my mac, since only two physical cores (with hyperthreading enabled)